## Existierende Datensätze finden und kombinieren

In [1]:
# ! pip install -U pandas openpyxl muclearn

### Umweltdaten des LfU

Link: https://www.lfu.bayern.de/umweltdaten/index.htm

Wir nehmen uns beispielhaft die Luftdaten PM10 für 2019 aus dem Messwertarchiv. Dann suchen wir uns eine Messstelle heraus und formatieren die Daten so, dass sie für ML benutzt werden können. Hierzu gehört die Umwandlung von Zeitstempeln, das Löschen fehlender Werte und die Umwandlung in tatsächliche Zahlen.

In [2]:
import pandas as pd

In [3]:
def fix_timestamp(ts: str):
    ts = ts.replace(" 24:00", " 00:00")
    ts = pd.to_datetime(ts, format="%d.%m.%Y %H:%M")
    if ts.hour == 0:
        ts = ts + pd.Timedelta(days=1)
    return ts

In [4]:
pm10 = pd.read_excel("../01_resources/PM10_2019.xlsx", header=1, parse_dates=["Zeitpunkt"]).dropna()

pm10["datetime"] = pm10.Zeitpunkt.apply(fix_timestamp)

pm10 = pm10.filter(axis=1, items=["München/Stachus", "datetime"])

pm10["München/Stachus"] = pm10["München/Stachus"].apply(pd.to_numeric, errors="coerce", downcast="integer")

pm10 = pm10.dropna()

pm10

,München/Stachus,datetime
0,524.0,2019-01-01 01:00:00
1,237.0,2019-01-01 02:00:00
2,83.0,2019-01-01 03:00:00
3,56.0,2019-01-01 04:00:00
4,46.0,2019-01-01 05:00:00
...,...,...
8755,38.0,2019-12-31 20:00:00
8756,38.0,2019-12-31 21:00:00
8757,47.0,2019-12-31 22:00:00
8758,57.0,2019-12-31 23:00:00


### Zeitbezogene Features mit muclearn berechnen

Das selbstentwickelte Paket muclearn (in der alpha) stellt diverse Hilfsfunktionen bereit.

Wir nutzen hier die im Subpaket date liegende Funktion calculate_datetime_features, die neben direkt aus Zeit und Datum ableitbaren werden auch Feiertage über das Package "holidays" sowie bayerische Schulferien, welche mithilfe von Camelot (https://camelot-py.readthedocs.io/en/master/) aus den Archiven der KMK (https://www.kmk.org/de/service/ferien/archiv-der-ferientermine.html) extrahiert wurden, enthält.

In [5]:
import muclearn.date

In [6]:
pm10_timed = muclearn.date.calculate_datetime_features(pm10)
pm10_timed = pm10_timed.set_index("datetime")
pm10_timed

,München/Stachus,hour,weekday,month,season,year,pre_holiday,holiday,post_holiday,ferien_by
datetime,,,,,,,,,,
2019-01-01 01:00:00,524.0,1,1,1,3,0,0,1,0,1
2019-01-01 02:00:00,237.0,2,1,1,3,0,0,1,0,1
2019-01-01 03:00:00,83.0,3,1,1,3,0,0,1,0,1
2019-01-01 04:00:00,56.0,4,1,1,3,0,0,1,0,1
2019-01-01 05:00:00,46.0,5,1,1,3,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...
2019-12-31 20:00:00,38.0,20,1,12,3,0,1,0,0,1
2019-12-31 21:00:00,38.0,21,1,12,3,0,1,0,0,1
2019-12-31 22:00:00,47.0,22,1,12,3,0,1,0,0,1


### Wetter-Daten vom DWD hinzufügen

https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/air_temperature/historical/

Wir suchen zuerst die ID der Münchner Station in der Beschreibung, laden dann den zugehörigen Ordner herunter und extrahieren die Datei mit den Stundenwerten.

In [7]:
weather = pd.read_csv("../01_resources/produkt_tu_stunde_19970701_20211231_03379.txt", sep=";")

weather["datetime"] = pd.to_datetime(weather.MESS_DATUM, format="%Y%m%d%H")

weather = weather.set_index("datetime")

weather = weather[pd.Timestamp(2019, 1, 1, 1):pd.Timestamp(2020, 1, 1)]

weather = weather.filter(items=["TT_TU"])

weather

,TT_TU
datetime,
2019-01-01 01:00:00,4.2
2019-01-01 02:00:00,4.0
2019-01-01 03:00:00,3.9
2019-01-01 04:00:00,4.0
2019-01-01 05:00:00,3.6
...,...
2019-12-31 20:00:00,2.2
2019-12-31 21:00:00,2.3
2019-12-31 22:00:00,2.2


In [8]:
pm10_full = pm10_timed.join(weather, on="datetime")

In [9]:
pm10_full

,München/Stachus,hour,weekday,month,season,year,pre_holiday,holiday,post_holiday,ferien_by,TT_TU
datetime,,,,,,,,,,,
2019-01-01 01:00:00,524.0,1,1,1,3,0,0,1,0,1,4.2
2019-01-01 02:00:00,237.0,2,1,1,3,0,0,1,0,1,4.0
2019-01-01 03:00:00,83.0,3,1,1,3,0,0,1,0,1,3.9
2019-01-01 04:00:00,56.0,4,1,1,3,0,0,1,0,1,4.0
2019-01-01 05:00:00,46.0,5,1,1,3,0,0,1,0,1,3.6
...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 20:00:00,38.0,20,1,12,3,0,1,0,0,1,2.2
2019-12-31 21:00:00,38.0,21,1,12,3,0,1,0,0,1,2.3
2019-12-31 22:00:00,47.0,22,1,12,3,0,1,0,0,1,2.2


In [11]:
pm10_full.to_csv("../01_resources/pm10_final.csv")